In [1]:
!pip install pandas_gbq

  Obtaining dependency information for pydata-google-auth>=1.5.0 from https://files.pythonhosted.org/packages/28/6b/3320c9ddbfc572108917e8432a07e8bd1e40054d94b5ad40c755afdc1160/pydata_google_auth-1.8.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core<3.0.0dev,>=2.10.2 from https://files.pythonhosted.org/packages/4d/ce/4fd62ea66b3508debc795e475336ce915929765870f0ad52328426ba016e/google_api_core-2.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth>=2.13.0 from https://files.pythonhosted.org/packages/d7/88/1826b0c047c48763b36ed854a984127b430a16b70003155d7b19975f1d59/google_auth-2.23.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-oauthlib>=0.7.0 from https://files.pythonhosted.org/packages/ce/33/a907b4b67245647746dde8d61e1643ef5d210c88e090d491efd89eff9f95/google_auth_oauthlib-1.1.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-bigquery!=2.4.*,<4.0.0dev,

In [4]:
import pandas as pd
from pandas_gbq import to_gbq
from google.cloud import bigquery
import pandas as pd
 
credentials = 'data-analyst-prj-bigquery-a71087fac92a.json'
bqclient = bigquery.Client.from_service_account_json(credentials)

#Crear el Schema destino
project_id = 'data-analyst-prj'
dataset_id = 'python_ecommerce_raw'

def fix_date_cols(df, tz='UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)
    return df

def load_csv(file, enc = 'utf-8'):
    full_url = 'ecommerce/' + file
    df = pd.read_csv(full_url, encoding=enc)
    
    if file == 'order_items.csv':
        df['shipping_limit_date']= pd.to_datetime(df['shipping_limit_date'])
        df = fix_date_cols(df)

    if file == 'order_reviews.csv':
        df['review_creation_date']= pd.to_datetime(df['review_creation_date'])
        df['review_answer_timestamp']= pd.to_datetime(df['review_answer_timestamp'])
        df = fix_date_cols(df)
        
    if file == 'orders.csv':
        df['order_purchase_timestamp']= pd.to_datetime(df['order_purchase_timestamp'])
        df['order_approved_at']= pd.to_datetime(df['order_approved_at'])
        df['order_delivered_carrier_date']= pd.to_datetime(df['order_delivered_carrier_date'])
        df['order_delivered_customer_date']= pd.to_datetime(df['order_delivered_customer_date'])
        df['order_estimated_delivery_date']= pd.to_datetime(df['order_estimated_delivery_date'])
        df = fix_date_cols(df)
        
    df = df.reset_index(drop=True)
    df.columns = map(lambda x: str(x).upper(), df.columns)
    
    table = file.lower().replace(".csv", '');
    table = table.replace(' ', '_');

    table_id = project_id + '.' + dataset_id + '.' + table
    job_config = bigquery.LoadJobConfig()
    job = bqclient.load_table_from_dataframe(df, table_id, job_config=job_config)  

    # Wait for the job to complete and then show the job results
    job.result()  

    # Read back the properties of the table
    table = bqclient.get_table(table_id)  
    print("Table:", table.table_id, "has", table.num_rows, "rows and", len(table.schema), "columns")       


In [5]:
load_csv('customers.csv')
load_csv('geolocation.csv')
load_csv('order_items.csv')
load_csv('order_payments.csv')
load_csv('order_reviews.csv')
load_csv('orders.csv')
load_csv('products.csv')
load_csv('sellers.csv')
load_csv('product_category_name_translation.csv')



Table: customers has 198882 rows and 5 columns
Table: geolocation has 2000326 rows and 5 columns
Table: order_items has 112650 rows and 7 columns
Table: order_payments has 103886 rows and 5 columns
Table: order_reviews has 99224 rows and 7 columns
Table: orders has 99441 rows and 8 columns
Table: products has 32951 rows and 9 columns
Table: sellers has 3095 rows and 4 columns
Table: product_category_name_translation has 71 rows and 2 columns
